# The San Pellegrino label moiré effect

Have you noticed the nice Moiré effect in the San Pellegrino label?

In this notebook, we are going to reverse engineer how this wavy pattern is being obtained.

# A quick glimpse at the label
I first gently cut the label from a San Pellegrino bottle and scanned it at 300dpi.

The notebook lives in a web browser, so we can easily display a number of web compatible formats (like this png file). 

Notice the wavy pattern which repeats itself. If you look closely it looks like it is obtained from the periodic repetition of a wavy curve and a line along two different directions. Hence the beating effect which corresponds to the fact that the two curves are out of phase (appears darker because more ink density) or in phase (appear lighter because the two curves are one upon the other).

In [ ]:
from IPython.display import Image as imdisplay
imfile = 'sanpe300.png'
imdisplay(imfile)

# Setup
## Let's get the image data
To get the image data as a an array of numbers we are going to use the pillow/PIL library. The advantage of using this library is that it yields an image object that has plenty of methods, so it is easy to obtain information on the channels in the image and some metadata like the dpi information (the label was scanned).

In [ ]:
from PIL import Image
from pprint import pprint

In [ ]:
im = Image.open(imfile)
print(type(im))

In [ ]:
print(im.getbands())
pprint(im.info)

To get to the data array itself, we can use some type casting of the image into a numpy array.

In [ ]:
import numpy as np
imarray = np.array(im)
imarray.shape

## Using xarray

Rather than sticking to an np.array, we are going to use a DataArray structure (from the xarray library). This allows to give name to the axes and also to work with meaningfull float coordinates (millimeter scale here) rather than the basic array indexing of the np.array.

This needs some additional work, but it pays for itself with the simplification of the subsequent code, and it is often important to work with the right scale (mm versus pixels).

In [ ]:
import xarray as xr

inch_mm = 25.4

def im2DataArray(im):
    imarray = np.array(im)
    dpi = im.info['dpi']
    xmax_mm, ymax_mm = im.width/dpi[0]*inch_mm,im.height/dpi[1]*inch_mm
    xs = np.linspace(0,xmax_mm,im.width)
    ys = np.linspace(0,ymax_mm,im.height)
    
    da = xr.DataArray(imarray,
                      coords = dict(y=ys,x=xs,channel = list(im.getbands())),
                      dims = ['y','x','channel'],
                      name='value')
    return da

In [ ]:
da = im2DataArray(im)
# to lower the memory footprint of imview () in some part of the notebook, I also
da_low = im2DataArray(Image.open('sanpe150.png'))

To display holoviews objects with the bokeh backend within deepnote, we need to define this show function. 

Now we are ready to benefit from the data array structure. We have setup everything that was needed to display the image along with a mm scale.

In [ ]:
import hvplot.xarray
imview = da_low.hvplot.rgb('x','y','value',bands = 'channel',flip_yaxis=True,data_aspect=1)
imview

## Cropping the image

We are first going to find what is the exact periodicity of the moiré pattern.

To achieve that we need to first crop a small $1cm^2$ data array in the full data array. We define the cropping box as a tuple and we also unpack the tuple as the x,y coordinates of the box. Notice how we can overlay two holoviews object with the * operator and get a layout with the + operator.

In [ ]:
import holoviews as hv 
xmin,ymin,xmax,ymax = box = (22,65,32,75)
square = hv.Rectangles(data=box).opts(fill_alpha=0.1)

crop = da.sel(x=slice(xmin,xmax),y=slice(ymin,ymax),channel = ['R','G','B'])
cropview = crop.hvplot.rgb('x','y','value',bands = 'channel',flip_yaxis=True,data_aspect=1)

(imview*square+cropview).opts(shared_axes=False).cols(1)

## Selection of the most contrasted channel

Rather than displaying the color image, we can display a layout of the images of each channel. 

It results that the channel with most contrast to distinguish between blue lines and the white background is the red channel (not the blue one!). In fact, there is almost as much blue in the white and in the line, hence the poor contrast of the blue channel.

In [ ]:
cropopts = dict(colorbar=False,cmap='gray',xaxis=None,yaxis=None,data_aspect=1,flip_yaxis=True)
crop.hvplot.image('x','y',**cropopts).layout('channel')

Another way to see that is to display the histograms of each channel.

In [ ]:
hist = crop.hvplot.hist('value',by='channel',alpha=0.5)
hist

# Translation between moiré patterns and between curves

## Translation between two moiré patterns

We select a second box upon the first one, and this time we only consider the R channel.

In [ ]:
boxes = dict()
boxes[0], boxes[1] = (22,65,32,75),(22,55,32,65)

square0 = hv.Rectangles(data=boxes[0]).opts(fill_alpha=0.1)
square1 = hv.Rectangles(data=boxes[1]).opts(fill_alpha=0.1)
selection = (imview*square0*square1)
selection

In [ ]:
def boxes2crop(boxes):
    crops = dict()
    for key,box in boxes.items():
        xmin,ymin,xmax,ymax = box
        sel = dict(x=slice(xmin,xmax),y=slice(ymin,ymax),channel = ['R'])
        crops[key] = da.sel(**sel).squeeze()
    return crops

crops = boxes2crop(boxes)


layout = hv.Layout([crop.hvplot.image('x','y',**cropopts,shared_axes=False) 
                    for k,crop in crops.items()])

layout

The two selected patterns are very similar but the second one appears to be shifted on the right. We are going to determine the exact subpixellic translation with a scikit-image registration algorithm.

In [ ]:
from skimage.registration import phase_cross_correlation
shift,_,_ = phase_cross_correlation(crops[0].data,crops[1].data,upsample_factor=10)
shift

We cancel this translation and make the two images match the most.
So we can check that the translation is correct by comparing the image difference. The hover tool is usefull to check the actual pixel values.

In [ ]:
from skimage.transform import SimilarityTransform, warp

t = SimilarityTransform(translation = (11.1,0))
h0 = hv.Image(crops[0].data).opts(tools = ['hover'])
im1translated = warp(crops[1].data,t,preserve_range=True)
h1translated = hv.Image(im1translated).opts(tools = ['hover'])

delta = crops[0].data-im1translated
hdelta = hv.Image(delta).opts(tools = ['hover'])
(h0+h1translated+hdelta).cols(2)

Now we know that the whole pattern is mainly invariant with respect to a translation of 10mm (i.e 118.1 pixels) vertically and 11.1 pixels horizontally (i.e 0.94mm).

In [ ]:
10*300/inch_mm, 11.1*inch_mm/300

## Translation between two wave curves

In the image above, we can count and see that there is room for a bit more than 16 almost horizontal wave curves in the 10mm vertical direction of the crop. Since the moiré pattern repeats itself for a translation of 10mm vertically and 0.94mm horizontally, there has to be 17 curves exactly in 10mm.

Thus the horizontal wave part pattern has to be invariant through a translation of vector translation_unit.

In [ ]:
translation_unit = np.array([-11.1,118.11])/16
translation_unit

To make the first subpattern pop, we can create a stack of 16 translated images and apply a median filtering on this stack.

In [ ]:
vignette = da.sel(channel='R').data
ims = []
for k in range(16):
    translation = k*translation_unit
    t = SimilarityTransform(translation = translation)
    ims.append(warp(vignette,t,preserve_range=True))

im_median = np.median(np.array(ims),axis=0)

Image.fromarray(im_median>128)

## Translation between line patterns
In the same way as before, we first determine the translation between the two moiré patterns that lie on a line orthogonal to the line pattern.
We find that these two boxes perfectly match.

In [ ]:
boxes = dict()
boxes[0], boxes[1] = (22,65,32,75),(34.085,40.915,44.085,50.915)

square0 = hv.Rectangles(data=boxes[0]).opts(fill_alpha=0.1)
square1 = hv.Rectangles(data=boxes[1]).opts(fill_alpha=0.1)
selection = (imview*square0*square1)
selection

In [ ]:
transl = np.array([-12.085,24.085])
vnorm = np.linalg.norm(transl)
angle = np.arccos(transl[1]/vnorm)
print('norm:',vnorm)
print('angle:',angle*180/np.pi)
14./10.*vnorm/np.cos(angle),14.5/10.*vnorm/np.cos(angle)

Since we count between 14 and 15 lines vertically in 10mm, there are 43 lines between the center of the two boxes (to persuade oneself, a small sketch comes handy).

As before, we make the hidden pattern appear with a median filtering of a stack of translated images.

In [ ]:
translation_unit2 = transl/43*300/inch_mm
ims = []
for k in range(43):
    translation = k*translation_unit2
    t = SimilarityTransform(translation = translation)
    ims.append(warp(vignette,t,preserve_range=True))

im_median2 = np.median(np.array(ims),axis=0)

Image.fromarray(im_median2>128)

# Reproducing the San Pellegrino label

To extract one single curve from the image we label the connected components with the measure.label function from scikit-image.

In [ ]:
from skimage import measure

crop = (im_median<128)[530:600,300:1000]
labels = measure.label(crop)
hlabel = hv.Image(labels).opts(cmap='nipy_spectral',data_aspect=0.1,frame_height=150)
hlabel

In [ ]:
subcrop = ((labels==1)*255).astype('uint8')
Image.fromarray(subcrop)

We now define a repeat function to translate and overlay an elementary image.

In [ ]:
def repeat(im,translation_unit,k_repeat):
    ims = []
    for k in range(-10,k_repeat):
        translation = k*translation_unit
        t = SimilarityTransform(translation = translation)
        ims.append(warp(im,t,preserve_range=True))

    return np.sum(np.array(ims),axis=0)

curveblack = np.concatenate((np.zeros((300,700)),subcrop)).astype('uint8')
curve_sum = repeat(curveblack,translation_unit,40)
Image.fromarray(curve_sum.astype('uint8'))

In [ ]:
lineblack = np.zeros(curve_sum.shape)
lineblack[131:133,:] = 255
rot = SimilarityTransform(rotation = -26.6*np.pi/180,translation=(0,100))
lineblack = warp(lineblack,rot,preserve_range = True)
line_sum = repeat(lineblack,translation_unit2,40)
Image.fromarray(line_sum.astype('uint8'))


These are the two patterns that were hidden in the San Pellegrino label. When we overlay one upon the other, we are back to the start with the iconic moiré pattern. 

Feel free to try other angle values, the aspect changes quite dramatically...

In [ ]:
Image.fromarray(~((curve_sum>128) | (line_sum>128)))